<a href="https://colab.research.google.com/github/whs1111/bert-version/blob/master/src/CLASS/bert4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
cd gdrive/My Drive/Colab_Notebooks/code


/content/gdrive/My Drive/Colab_Notebooks/code


In [3]:
#@title
!git clone https://github.com/NVIDIA/apex.git
%cd apex
!pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" .
%cd ..

import torch
import apex
from fastai.text import *
import datetime
run_start_time = datetime.datetime.today().strftime('%Y-%m-%d_%H-%M-%S')

LOG_PATH=Path('logs/')  
MODEL_PATH=Path('models/') 

if not LOG_PATH.exists():
  LOG_PATH.mkdir()
import logging

args = {
    "run_text": "my_test",
    "max_seq_length": 30,
    "do_lower_case": True,
    "train_batch_size": 16,
    "learning_rate": 6e-5,
    "num_train_epochs": 12.0,
    "warmup_proportion": 0.002,
    "local_rank": -1,
    "gradient_accumulation_steps": 1,
    "fp16": True,
    "loss_scale": 128
}

logfile = str(LOG_PATH/'log-{}-{}.txt'.format(run_start_time, args["run_text"]))

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
    datefmt='%m/%d/%Y %H:%M:%S',
    handlers=[
        logging.FileHandler(logfile),
        logging.StreamHandler(sys.stdout)
    ])

logger = logging.getLogger()

device = torch.device('cuda')

if torch.cuda.device_count() > 1:
    multi_gpu = True
else:
    multi_gpu = False

fatal: destination path 'apex' already exists and is not an empty directory.
/content/gdrive/My Drive/Colab_Notebooks/code/apex
/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:243: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-z5a11094
Created temporary directory: /tmp/pip-req-tracker-qobad8fc
Created requirements tracker '/tmp/pip-req-tracker-qobad8fc'
Created temporary directory: /tmp/pip-install-xejocou7
Processing /content/gdrive/My Drive/Colab_Notebooks/code/apex
  Created temporary directory: /tmp/pip-req-build-c99c6d9o
  Added file:///content/gdrive/My%20Drive/Colab_Notebooks/code/apex to build tracker '/tmp/pip-req-tracker-qobad8fc'
    Running setup.py (path:/tmp/pip-req-build-c99c6d9o/setup.py) egg_info for package from file:///content/gdrive/My%20Drive/Colab_Notebooks/code/apex
 

In [4]:
!pip install fast-bert

     |████████████████████████████████| 51kB 4.4MB/s 
     |████████████████████████████████| 163kB 17.2MB/s 
     |████████████████████████████████| 225kB 14.0MB/s 
     |████████████████████████████████| 1.0MB 44.0MB/s 
     |████████████████████████████████| 655kB 42.2MB/s 
  Created wheel for regex: filename=regex-2019.8.19-cp36-cp36m-linux_x86_64.whl size=609236 sha256=6ced8b130974c2677a0a255969be5efffe3038b66f92831c5ee6e185f089ca83
  Stored in directory: /root/.cache/pip/wheels/90/04/07/b5010fb816721eb3d6dd64ed5cc8111ca23f97fdab8619b5be
Successfully built regex


In [0]:
from fast_bert.data import *
from fast_bert.learner_cls import *
from fast_bert.metrics import *


In [0]:
DATA_PATH = Path('bert-version/src/DATA/')     
LABEL_PATH = Path('bert-version/src/DATA/')  

BERT_PRETRAINED_MODEL = "bert-base-uncased"

args["do_lower_case"] = True
args["train_batch_size"] = 16
args["learning_rate"] = 6e-5
args["max_seq_length"] = 140
args["fp16"] = True

label_cols = ["GoodsServices",
        "SearchAndRescue",
        "InformationWanted",
        "Volunteer",
        "FundRaising",
        "Donations",
        "MovePeople",
        "FirstPartyObservation",
        "ThirdPartyObservation",
        "Weather",
        "EmergingThreats",
        "NewSubEvent",
        "MultimediaShare",
        "ServiceAvailable",
        "Factoid",
        "Official",
        "CleanUp",
        "Hashtags",
        "ContextualInformation",
        "News",
        "Advice",
        "Sentiment",
        "Discussion",
        "Irrelevant",
        "OriginalEvent"]
import logging

logger = logging.getLogger()
device_cuda = torch.device("cuda")

OUTPUT_DIR = Path('bert-version/src/DATA/') 
from fast_bert.data_cls import BertDataBunch

databunch = BertDataBunch(DATA_PATH, LABEL_PATH,
                          tokenizer='bert-base-uncased',
                          train_file='train.csv',
                          val_file='valid.csv',
                          label_file='labels.csv',
                          text_col='text',
                          label_col=label_cols,
                          batch_size_per_gpu=16,
                          max_seq_length=140,
                          multi_gpu=True,
                          multi_label=True,
                          model_type='bert')
metrics = [{'name': 'accuracy', 'function': accuracy_multilabel}]
logger.info(args)
learner = BertLearner.from_pretrained_model(
						databunch,
						pretrained_path='bert-base-uncased',
						metrics=metrics,
						device=device_cuda,
						logger=logger,
						output_dir=OUTPUT_DIR,
						finetuned_wgts_path=None,
						warmup_steps=500,
						multi_gpu=True,
						is_fp16=True,
						multi_label=True,
						logging_steps=50)
learner.fit(4, lr=args['learning_rate'], schedule_type="warmup_linear")

09/03/2019 08:42:49 - INFO - pytorch_transformers.tokenization_utils -   loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /root/.cache/torch/pytorch_transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
09/03/2019 08:43:05 - INFO - root -   Writing example 0 of 15051
09/03/2019 08:43:08 - INFO - root -   Writing example 10000 of 15051
09/03/2019 08:43:09 - INFO - root -   Saving features into cached file bert-version/src/DATA/cache/cached_bert_train_multi_label_140
09/03/2019 08:43:16 - INFO - root -   Writing example 0 of 3613
09/03/2019 08:43:17 - INFO - root -   Saving features into cached file bert-version/src/DATA/cache/cached_bert_dev_multi_label_140
09/03/2019 08:43:18 - INFO - root -   {'run_text': 'my_test', 'max_seq_length': 140, 'do_lower_case': True, 'train_batch_size': 16, 'learning_rate': 6e-05, 'num_train_epochs': 12.0, 'warmu

In [0]:
learner.save_and_reload('/content/gdrive/My Drive/Colab_Notebooks/code/bert-version/src/DATA/', "bert-model")
from fast_bert.prediction import BertClassificationPredictor

predictor = BertClassificationPredictor(model_path='bert-version/src/DATA/bert-model.bin', pretrained_path="bert-base-uncased", 
                                        label_path='/content/gdrive/My Drive/Colab_Notebooks/code/bert-version/src/DATA/', multi_label=True)

# Single prediction
single_prediction = predictor.predict("just get me result for this text")
print(single_prediction)

# Batch predictions
# texts = [
#   "this is the first text",
#   "this is the second text"
# ]

# multiple_predictions = predictor.predict(texts)


07/31/2019 14:04:06 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /root/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
07/31/2019 14:04:06 - INFO - pytorch_pretrained_bert.modeling -   extracting archive file /root/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /tmp/tmpw726d29o
07/31/2019 14:04:09 - INFO - pytorch_pretrained_bert.modeling -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size